## 1. Example of steaming (CEPE courses)

* source : https://medium.com/expedia-group-tech/apache-spark-structured-streaming-input-sources-2-of-6-6a72f798838c. This source is very intresting about differents input. I took one with local files but there are many others
* data : all data is in my spark data set directory :"data_spark/stocks/cepe". Only take 4 campanies
* Upload to databrick : homepage -> data import -> take 2 campagnies first (GOOGLE & Amazon) -> files will be in "/FileStore/tables/". Do this 2 files by 2 files. Pay attention to filename. Databricks rename your file
* If you want to delete imported csv files, run this code : dbutils.fs.rm("/FileStore/tables/NKE_2006_01_01_to_2018_01_01.csv"). Again, pay attention to file name and the boolean should return True

In [0]:
dbutils.fs.ls("FileStore/tables")

dbutils.fs.rm("FileStore/tables/",True)

Out[115]: True

In [0]:
# Only GOOGLE & Amazon

# on importe en static juste pour mieux comprendre les data, we can also take a look of 
static = (spark.read.format("csv")
          .option("header", "true")
          .option("inferschemas", "true")
          .load("/FileStore/tables") # pay attention to path, there must be a / before FileStore
         
         )
df_schema = static.schema # this is necessary because in readStream we have to specify a schema 
static.show() 
static.printSchema() # first print schemas, everything is string type
static.count() # 6038

+----------+------+------+------+------+--------+-----+
|      Date|  Open|  High|   Low| Close|  Volume| Name|
+----------+------+------+------+------+--------+-----+
|2006-01-03|211.47|218.05|209.32|217.83|13137450|GOOGL|
|2006-01-04|222.17| 224.7|220.09|222.84|15292353|GOOGL|
|2006-01-05|223.22| 226.0|220.97|225.85|10815661|GOOGL|
|2006-01-06|228.66|235.49|226.85|233.06|17759521|GOOGL|
|2006-01-09|233.44|236.94| 230.7|233.68|12795837|GOOGL|
|2006-01-10|232.44|235.36|231.25|235.11| 9104719|GOOGL|
|2006-01-11|235.87|237.79|234.82|236.05| 9008664|GOOGL|
|2006-01-12| 237.1|237.73|230.98|232.05|10125212|GOOGL|
|2006-01-13|232.39|233.68|231.04|233.36| 7660220|GOOGL|
|2006-01-17|231.76|235.18| 231.5|233.79| 8335300|GOOGL|
|2006-01-18|223.87|228.91|221.85|222.68|20511176|GOOGL|
|2006-01-19|225.81|226.97|216.72|218.44|14539830|GOOGL|
|2006-01-20|219.57|220.24|197.57|199.93|41182889|GOOGL|
|2006-01-23|203.89|214.41|203.07|213.96|22768073|GOOGL|
|2006-01-24|218.23| 222.7|217.46|221.74|15468453

In [0]:
# Defin a Schema. this is optionnal beacause without this definition all works
schema = StructType((
  StructField("Date", StringType(), True),
  StructField("Open", DoubleType(), True),
  StructField("High", DoubleType(), True),
  StructField("Low", DoubleType(), True),
  StructField("Close", DoubleType(), True),
#   StructField("Adjusted Close", DoubleType(), True),
  StructField("Volume", DoubleType(), True),
  StructField("Name", StringType(), True)
))


In [0]:
# Create Streaming DataFrame by reading data from File Source.
# In readStream, there is no inferSchema option in readStream, and we must specify a schema
initDF = (spark.readStream
       .format("csv")
       .option("maxFilesPerTrigger", 1) # This will read maximum of 1 file per mini batch. (if 2) However, it can read less than 2 files.
       .option("header", "true")
#        .option("inferSchema", "true") # there is no inferSchema option in readStream
       .schema(df_schema) # and we must specify a schema    
       .load("/FileStore/tables/")          
      )

# attention : we can not add a action to steam dataframe

In [0]:
# Then we specify a query
from pyspark.sql.functions import year, month, max, avg
stockDf =(initDF.withColumn("year", year("Date"))
 .withColumn("month", month("Date"))
 .groupby("Name", "year", "month")
 .agg(avg("Volume").alias("avg_volume"))
#  .orderBy("Name", "year", "month") # Attention, Sorting is not supported on streaming DataFrames/Datasets 
)

In [0]:
# Then we execute this query in streaming mode in memory
(stockDf
 .writeStream
 .outputMode("update") # or "complete", "Append" output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark
 .format("memory")
 .queryName("test")
 .start()
)

* Pour afficher la visualisation, cliquer sur "+" à coté de onglet table et choisir la représentation : scatter, x=year, y=avg, group = name
* Puis exécuter plusieurs fois la requête. Ajouter les deux autres fichiers, réexécuter plusieurs fois la requêtes

In [0]:
%sql  SELECT * FROM test order by Name, year, month

In [0]:
# from pyspark.sql.functions import bu
from time import sleep
for x in range(25):
    spark.sql("SELECT Name, Year, sum(avg_volume)) FROM test group by Name, Year order by Name, Year").show(200) # table name "test" because.queryName("test"), attention aussi a nb de ligne affiche
    sleep(4)

+-----+----+-------------------------+
| Name|Year|round(sum(avg_volume), 2)|
+-----+----+-------------------------+
| AABA|2006|           2.8846436558E8|
| AABA|2007|           3.1964592877E8|
| AABA|2008|           4.0840655454E8|
| AABA|2009|           2.8069592001E8|
| AABA|2010|           2.6886491367E8|
| AABA|2011|           3.3203429808E8|
| AABA|2012|            2.238929468E8|
| AABA|2013|           2.1742498673E8|
| AABA|2014|           2.8891486406E8|
| AABA|2015|           1.8827500652E8|
| AABA|2016|           1.5961967619E8|
| AABA|2017|           1.3267999895E8|
| AAPL|2006|          2.59492653268E9|
| AAPL|2007|          2.95587563613E9|
| AAPL|2008|          3.38745408604E9|
| AAPL|2009|          1.71177135312E9|
| AAPL|2010|          1.81080700077E9|
| AAPL|2011|           1.4775206632E9|
| AAPL|2012|          1.58702389042E9|
| AAPL|2013|          1.22188505516E9|
| AAPL|2014|           7.6008627299E8|
| AAPL|2015|            6.218254999E8|
| AAPL|2016|           4.

In [0]:
# import amazone et google
dbutils.fs.ls("FileStore/tables")

# remove all files in a directory
dbutils.fs.rm("FileStore/tables/",True)

# remove a specific file in a directory
dbutils.fs.rm('FileStore/tables/XOM_2006_01_01_to_2018_01_01.csv',True)

Out[43]: True

In [0]:
# Only GOOGLE & Amazon

# on importe en static juste pour mieux comprendre les data, we can also take a look of 
static = (spark.read.format("csv")
          .option("header", "true")
          .option("inferschemas", "true") 
          .load("/FileStore/tables/") # pay attention to path, there must be a / before FileStore
         )
dataSchema = static.schema

static.count() # 6038

static.printSchema() # first print schemas, everything is string type

Out[2]: <bound method DataFrame.printSchema of DataFrame[Date: string, Open: string, High: string, Low: string, Close: string, Volume: string, Name: string]>

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
# Define Schema
schema = StructType((
  StructField("Date", StringType(), True),
  StructField("Open", DoubleType(), True),
  StructField("High", DoubleType(), True),
  StructField("Low", DoubleType(), True),
  StructField("Close", DoubleType(), True),
#   StructField("Adjusted Close", DoubleType(), True),
  StructField("Volume", DoubleType(), True),
  StructField("Name", StringType(), True)
))


# Extract the Name of the stock from the file name.
# def getFileName():
#   file_name = reverse(split(input_file_name(), "/")).getItem(0)
#   split(file_name, "_").getItem(0)



In [0]:
# Create Streaming DataFrame by reading data from File Source.
initDF =(spark.readStream
         .format("csv")
         .option("maxFilesPerTrigger", 1) # This will read maximum of 1 file per mini batch. (if 2) However, it can read less than 2 files.
         .option("header", True)
         .option("path", "/FileStore/tables/")
         .schema(schema).load()
#          .withColumn("Name",input_file_name())
#          .withColumn("Name", getFileName)
        )


In [0]:
stockDf = (initDF
         .groupBy(col("Name"), year(col("Date")).alias("Year"))
         .agg(max("High").alias("Max"))
          )


In [0]:
initDF

Out[51]: DataFrame[Date: string, Open: double, High: double, Low: double, Close: double, Volume: double, Name: string]

In [0]:
stockDf

Out[52]: DataFrame[Name: string, Year: int, Max: double]

In [0]:
# Then we execute this query in streaming mode in memory
activityQuery = (stockDf
  .writeStream
  .outputMode("update") # Try "append", "update" and "complete" mode.
  .option("truncate", False)
  .option("numRows", 10)
  .format("memory")
  .queryName("test")
  .start()
#   .awaitTermination() # do not put this line
)

Pour afficher la visualisation, cliquer sur "+" à coté de onglet table et choisir la représentation. Puis exécuter plusieurs fois la requête. Ajouter les deux autres fichiers, réexécuter plusieurs fois la requêtes

In [0]:
from time import sleep
for x in range(25):
    spark.sql("SELECT * FROM test order by Name, Year").show(80) # table name "test" because.queryName("test"), attention aussi a nb de ligne affiche
    sleep(8)

+-----+----+-------+
| Name|Year|    Max|
+-----+----+-------+
| AMZN|2006|  48.58|
| AMZN|2007| 101.09|
| AMZN|2008|  97.43|
| AMZN|2009| 145.91|
| AMZN|2010| 185.65|
| AMZN|2011| 246.71|
| AMZN|2012| 264.11|
| AMZN|2013| 405.63|
| AMZN|2014| 408.06|
| AMZN|2015| 696.44|
| AMZN|2016| 847.21|
| AMZN|2017|1213.41|
|GOOGL|2006| 256.76|
|GOOGL|2007| 373.99|
|GOOGL|2008| 349.03|
|GOOGL|2009| 313.31|
|GOOGL|2010| 315.74|
|GOOGL|2011|  323.7|
|GOOGL|2012| 387.58|
|GOOGL|2013| 561.06|
|GOOGL|2014| 615.05|
|GOOGL|2015| 798.69|
|GOOGL|2016|  839.0|
|GOOGL|2017|1086.49|
|  IBM|2006|  97.88|
|  IBM|2007| 121.46|
|  IBM|2008| 130.93|
|  IBM|2009| 132.85|
|  IBM|2010| 147.53|
|  IBM|2011|  194.9|
|  IBM|2012| 211.79|
|  IBM|2013|  215.9|
|  IBM|2014| 199.21|
|  IBM|2015|  176.3|
|  IBM|2016| 169.95|
|  IBM|2017| 182.79|
|  NKE|2006|  12.65|
|  NKE|2007|  16.98|
|  NKE|2008|  17.65|
|  NKE|2009|  16.66|
|  NKE|2010|  23.12|
|  NKE|2011|  24.56|
|  NKE|2012|   28.7|
|  NKE|2013|  40.13|
|  NKE|2014| 

### 2. Example from the book with databricks data uploading

In [0]:
static = spark.read.json("/FileStore/tables/")
dataSchema = static.schema

In [0]:
static.count()

Out[39]: 168104

In [0]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
    .json("/FileStore/tables/")

In [0]:
# in Python
activityCounts = streaming.groupBy("gt").count()

In [0]:
activityQuery = activityCounts.writeStream.queryName("activity_counts")\
.format("memory").outputMode("complete")\
.start()

In [0]:
from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)

+----+-----+
|  gt|count|
+----+-----+
|null| 3020|
+----+-----+

+----+-----+
|  gt|count|
+----+-----+
|null| 6040|
+----+-----+

+----+-----+
|  gt|count|
+----+-----+
|null| 6040|
+----+-----+

+----+-----+
|  gt|count|
+----+-----+
|null| 9060|
+----+-----+

+----+-----+
|  gt|count|
+----+-----+
|null| 9060|
+----+-----+

